In [23]:
%reload_kedro

[05/07/24 11:23:56] INFO     Resolved project path as:                                              ]8;id=10401;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=939070;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro\ipython\__init__.py#164\164]8;;\
                             C:\Users\9509298u\Documents\GitHub\OC_Projects\ocp7-scoring-model-clou                
                             d.                                                                                    
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=758077;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=218357;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=729311;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=503486;file://C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///C:/Users/9509298u/Documents/GitHub/OC_Projects/ocp                           
                             7-scoring-model-cloud/mlruns'                                                         

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 get_ipython().run_line_magic('reload_kedro', '')                                             │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\IPyth │
│ on\core\interactiveshell.py:2480 in run_line_magic                                               │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro │
│ \ipython\__init__.py:97 in magic_reload_kedro                                                    │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro │
│ \ipython\__init__.py:122 in reload_kedro                                                         │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\kedro │
│ \framework\session\session.py:243 in load_context                                                │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_hooks.py:493 in __call__                                                                      │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_manager.py:115 in _hookexec                                                                   │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_manager.py:457 in traced_hookexec                                                             │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_result.py:114 in get_result                                                                   │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_result.py:76 in from_call                                                                     │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_manager.py:454 in <lambda>                                                                    │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scoring-model-cloud\lib\site-packages\plugg │
│ y\_callers.py:113 in _multicall                                                                  │
│                                                                                                  │
│ C:\Users\9509298u\AppData\Local\miniconda3\envs\ocp7-scorin

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import missingno as msno


In [ ]:
dataset_type = "" #change for "_debug" to load the debug dataset

df_train = catalog.load("application_train" + dataset_type)

In [ ]:
df_train.head()

In [ ]:
df_train.columns.tolist()

In [ ]:
df_train.head()

In [ ]:
# Find correlations with the target and sort
correlations = df_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [13]:
invididual_variables = ["DAYS_BIRTH", "CODE_GENDER", "NAME_EDUCATION_TYPE"]
work_variables = ["DAYS_EMPLOYED", "OCCUPATION_TYPE", "ORGANIZATION_TYPE", "AMT_INCOME_TOTAL"]
loan_variables = ["AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]
ext_variables = ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]
variables_interest = invididual_variables + work_variables + loan_variables + ext_variables
# liés à l'emprunt, liées à la personne, liées au travail, liées aux autres sources financières

nous voulons retenir les variables les plus importantes pour l'analyse 
nous pouvons observer les variables les mieux corrélées au défaut de crédit

In [15]:
df_train[["SK_ID_CURR"] + variables_interest]

,SK_ID_CURR,DAYS_BIRTH,CODE_GENDER,NAME_EDUCATION_TYPE,DAYS_EMPLOYED,OCCUPATION_TYPE,ORGANIZATION_TYPE,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,100002,-9461,M,Secondary / secondary special,-637,Laborers,Business Entity Type 3,202500.0,406597.5,24700.5,351000.0,0.083037,0.262949,0.139376
1,100003,-16765,F,Higher education,-1188,Core staff,School,270000.0,1293502.5,35698.5,1129500.0,0.311267,0.622246,NaN
2,100004,-19046,M,Secondary / secondary special,-225,Laborers,Government,67500.0,135000.0,6750.0,135000.0,NaN,0.555912,0.729567
3,100006,-19005,F,Secondary / secondary special,-3039,Laborers,Business Entity Type 3,135000.0,312682.5,29686.5,297000.0,NaN,0.650442,NaN
4,100007,-19932,M,Secondary / secondary special,-3038,Core staff,Religion,121500.0,513000.0,21865.5,513000.0,NaN,0.322738,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,-9327,M,Secondary / secondary special,-236,Sales staff,Services,157500.0,254700.0,27558.0,225000.0,0.145570,0.681632,NaN
307507,456252,-20775,F,Secondary / secondary special,365243,NaN,XNA,72000.0,269550.0,12001.5,225000.0,NaN,0.115992,NaN
307508,456253,-14966,F,Higher education,-7921,Managers,School,153000.0,677664.0,29979.0,585000.0,0.744026,0.535722,0.218859
307509,456254,-11961,F,Secondary / secondary special,-4786,Laborers,Business Entity Type 1,171000.0,370107.0,20205.0,319500.0,NaN,0.514163,0.661024


In [16]:
from src.ocp7_scoring_model_cloud.pipelines.preprocessing.nodes import (
    preprocess_application_train)

In [17]:
preprocess_train_df = preprocess_application_train(df_train)

Train samples: 307511


In [19]:
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
preprocess_train_df.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Stonebrick,WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,False,True,False,True,False,0.067329,0.498036,202500.0,0.121978,0.060749
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,False,False,False,True,False,0.070862,0.208736,135000.0,0.132217,0.027598
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,False,False,False,False,False,0.011814,0.500000,67500.0,0.100000,0.050000
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,False,False,False,False,False,0.159905,0.431748,67500.0,0.219900,0.094941
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,False,False,False,False,False,0.152418,0.236842,121500.0,0.179963,0.042623


In [21]:
preprocess_train_df[individual_variables].head()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 preprocess_train_df[individual_variables].head()                                             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'individual_variables' is not defined